In [189]:
#检测和处理大整数
import re
def convert_large_integers(line):
    for match in re.findall(r'\b\d+\b', line):
        value = int(match)
        if value > 2**53 - 1:
            line = line.replace(match, f'BigInt("{match}")')
    return line

In [190]:
#处理变量赋值
def convert_assignment(line):
    
    return line




In [191]:
#处理注释
def convert_comments(line):
    
    return line.replace("#", "//")


In [192]:
#处理打印输出
def convert_print(line):
    
    return line.replace("print(", "console.log(")


In [193]:
#处理基本的数学运算
def convert_operations(line):
    
    if "**" in line:
        line = line.split("**")
        return "Math.pow(" + line[0] + ", " + line[1] + ")"
    return line

In [194]:
#处理空白行
def convert_python_to_js(python_code):
    js_code = ""
    for line in python_code.split("\n"):
        line = line.strip()
        if line == "" or line.startswith("#"):
            js_code += line + "\n"
            continue
        if line.startswith("print"):
            js_code += convert_print(line) + "\n"
        elif "=" in line:
            js_code += convert_assignment(line) + "\n"
        else:
            js_code += convert_operations(line) + "\n"
    return js_code


In [195]:
#处理if-else
def convert_if_else(statement):
    # 将语句分割成组成部分
    components = statement.split()
    # 第一个组成部分是 'if'，'elif'，或 'else'
    keyword = components[0]
    # 如果关键词是 'elif'，转换为 'else if'
    if keyword == "elif":
        keyword = "else if"
    # 如果关键词是 'else'，只返回 'else {'
    elif keyword == "else":
        return "else {"

    # 剩下的部分是条件
    condition = " ".join(components[1:]).strip(':')
    # 将条件包裹在括号中
    condition = "(" + condition + ")"
    # 重构语句
    converted_statement = keyword + " " + condition + " {"
    return converted_statement
def convert_if(statement):
    condition = statement[3:].strip(':')
    return "if (" + condition + ")"

def convert_elif(statement):
    condition = statement[5:].strip(':')
    return "else if (" + condition + ")"





In [196]:
#处理for循环
def convert_for_loop(statement):
    components = statement.split()
    variable = components[1]
    if "range" in statement:
        range_args = components[3].split("(")[1].split(")")[0].split(",")
        if len(range_args) == 1: 
            start = "0"
            end = range_args[0]
        elif len(range_args) == 2: 
            start = range_args[0]
            end = range_args[1]
        converted_statement = f"for (let {variable} = {start}; {variable} < {end}; {variable}++)" + " {"
    else:
        iterable = components[3].strip(":")
        converted_statement = f"for (let {variable} of {iterable})" + " {"
    return converted_statement



In [197]:
#处理wile循环
def convert_while(statement):
    condition = statement[5:].strip(':')
    return "while (" + condition + ") {"



In [198]:
#检测行缩进处理嵌套结构
def get_indent(line):
    return len(line) - len(line.lstrip())


In [199]:
#处理函数定义
def convert_function_def(line):
    line = line.replace("def ", "function ")
    line = line.replace(":", " {")
    return line


In [200]:
#处理类定义
def convert_class_def(line):
    line = line.replace(":", " {")
    return line


In [201]:
#处理方法定义
def convert_method_def(line):
    line = line.replace("def ", "")
    line = line.replace("(self", "(").replace("(self,", "(")
    line = line.replace(":", " {")
    line = line.replace("self.", "this.")
    return line

In [202]:
#替换
def convert_self_to_this(line):
    return line.replace("self.", "this.")


In [203]:
#处理类构造函数
def convert_constructor(statement):
    parts = statement.split()
    parameters = parts[2].strip("():")  
    parameters = parameters.split(",")
    parameters = [param.strip() for param in parameters if param.strip() and param.strip() != "self"]  #删掉self
    constructor_line = "constructor({})".format(", ".join(parameters))
    constructor_line += " {"  
    return constructor_line






In [204]:
#主函数
#添加分号，处理行内的多个语句
def convert_python_to_js(python_code):
    js_code = ""
    lines = python_code.split("\n")
    lines = [convert_comments(line) for line in lines]
    lines = [convert_large_integers(line) for line in lines]
    current_indent = 0
    in_class_def = False
    close_brackets = 0

    for line in lines:
        indent = get_indent(line)
        line = convert_self_to_this(line)

        if indent < current_indent:
            close_brackets += current_indent - indent

        current_indent = indent

        for statement in line.strip().split(";"):
            statement = statement.strip()

            
            if close_brackets > 0:
                js_code += "}" * (close_brackets // 4) + "\n"
                close_brackets = 0

            if statement == "" or statement.startswith("#"):
                js_code += statement + "\n"
                continue
            if statement.startswith("print"):
                js_code += " " * indent + convert_print(statement) + ";\n"
            elif "=" in statement and "for" not in statement and "if" not in statement and "while" not in statement:
                js_code += " " * indent + convert_assignment(statement) + ";\n"
            elif statement.startswith("if"):
                js_code += " " * indent + convert_if(statement) + " {\n"
            elif statement.startswith("elif"):
                js_code += " " * indent + convert_elif(statement) + " {\n"
            elif statement.startswith("else"):
                js_code += " " * indent + "else {\n"
            elif statement.startswith("for"):
                js_code += " " * indent + convert_for_loop(statement) + "\n"
            elif statement.startswith("while"):
                js_code += " " * indent + convert_while(statement) + "\n"
            elif statement.startswith("def"):
                if in_class_def:
                    if "__init__" in statement:
                        js_code += " " * indent + convert_constructor(statement) + "\n"
                    else:
                        js_code += " " * indent + convert_method_def(statement) + "\n"
                else:
                    js_code += " " * indent + convert_function_def(statement) + "\n"
            elif statement.startswith("class"):
                js_code += " " * indent + convert_class_def(statement) + "\n"
                in_class_def = True
            else:
                js_code += " " * indent + convert_operations(statement) + ";\n"

    if current_indent > 0 or close_brackets > 0:
        js_code += "}" * (current_indent + close_brackets)/4 + "\n"
    return js_code







In [205]:
python_code = """
# Assignment
x = 10
y = 20.5
z = x + y

# Print
print("Hello, world!")

# Control structures
if x > y:
    print("x is greater than y")
elif x < y:
    print("x is less than y")
else:
    print("x is equal to y")

for i in range(10):
    print(i)

i = 0
while i < 10:
    print(i)
    i += 1

# Function definition
def add(a, b):
    return a + b

# Class definition
class MyClass:
    def __init__(self, x):
        self.x = x
    def print_x(self):
        print(self.x)

# Instantiation
obj = MyClass(10)
obj.print_x()

# Large integer
large_int = 12345678901234567890

"""

js_code = convert_python_to_js(python_code)
print(js_code)


// Assignment;
x = 10;
y = 20.5;
z = x + y;

// Print;
console.log("Hello, world!");

// Control structures;
if (x > y) {
    console.log("x is greater than y");
}
else if (x < y) {
    console.log("x is less than y");
}
else {
    console.log("x is equal to y");
}

for (let i = 0; i < 10; i++) {
    console.log(i);
}

i = 0;
while ( i < 10) {
    console.log(i);
    i += 1;
}

// Function definition;
function add(a, b) {
    return a + b;
}

// Class definition;
class MyClass {
    constructor(x) {
        this.x = x;
}
    print_x() {
        console.log(this.x);
}}

// Instantiation;
obj = MyClass(10);
obj.print_x();

// Large integer;
large_int = BigInt("12345678901234567890");



